# 2 Домашнее задание от Karpov Courses 

## 1 Задание 
#### Теперь поработаем с данными о поездках на такси в Нью-Йорке, где также имеется информация о погодных условиях и выходных днях.  

#### Но перед тем, как приступать к работе, давайте загрузим данные. 

На LMS путь к данным лежит в переменной path_to_file (условно, path_to_file = "/some/path/to/file.csv"). 

Импортируйте библиотеку pandas, а затем загрузите данные по вышеупомянутой ссылке. 

Результат сохраните в переменную taxi.

Проверьте, сколько всего строк и столбцов имеется в датете.



In [27]:
import pandas as pd

In [28]:
taxi = pd.read_csv("2_taxi_nyc.csv")

In [29]:
print(taxi.shape)

(29101, 14)


#### Давайте посмотрим на типы колонок. 

Все ли из них считались правильно? 
В качестве ответа выберите тип, преобладающий в датасете.

In [30]:
print(taxi.dtypes)

pickup_dt        object
pickup_month     object
borough          object
pickups           int64
hday             object
spd             float64
vsb             float64
temp            float64
dewp            float64
slp             float64
pcp 01          float64
pcp 06          float64
pcp 24          float64
sd              float64
dtype: object


In [31]:
type_counts = taxi.dtypes.value_counts()
print(type_counts)

float64    9
object     4
int64      1
Name: count, dtype: int64


## 2 Задание 
#### В названиях столбцов pcp 01, pcp 06, pcp 24 встречается пробел. 
Это не очень удобно, если в дальнейшем вы планируете обращаться к колонкам через точку, без использования кавычек и скобочек. 
Замените пробел в названиях на знак нижнего подчеркивания. 
Данные сохранены в переменную taxi. Данные должны быть изменены в этом же датафрейме


In [32]:

taxi = taxi.rename(columns={
    'pcp 01': 'pcp_01',
    'pcp 06': 'pcp_06', 
    'pcp 24': 'pcp_24'
})


print(taxi.columns)

Index(['pickup_dt', 'pickup_month', 'borough', 'pickups', 'hday', 'spd', 'vsb',
       'temp', 'dewp', 'slp', 'pcp_01', 'pcp_06', 'pcp_24', 'sd'],
      dtype='object')


## 3 Задание 
#### Вас попросили узнать, сколько записей (строк) в датафрейме относятся к району Манхэттен (Manhattan). 
Получить ответ на данный вопрос можно было бы, например, с помощью следующей команды:

А если мы хотим посмотреть, сколько раз встречается каждый из районов? Неужели придется использовать подобную конструкцию для каждого уровня переменной?

Сколько раз в данных встречается район Бруклин (Brooklyn)?

In [33]:
taxi.borough.value_counts()

borough
Bronx            4343
Brooklyn         4343
EWR              4343
Manhattan        4343
Queens           4343
Staten Island    4343
Name: count, dtype: int64

## 4 Задание 
#### Следующая задача: выяснить, из какого района было совершено наибольшее количество поездок за весь период.

Для начала, посчитайте общее количество поездок (pickups), без группировки.

In [34]:
total_pickups = taxi.pickups.sum()

print(total_pickups)

14265773


А теперь сгруппируйте данные по району (borough) и укажите, из какого пункта было совершено больше всего поездок.

In [35]:
borough_totals = taxi.groupby('borough')['pickups'].sum()
max_borough = borough_totals.idxmax()
max_pickups = borough_totals.max()

print(f"Больше всего поездок было из {max_borough}: {max_pickups} поездок")

Больше всего поездок было из Manhattan: 10367841 поездок


## ⭐️Задание со звёздочкой!⭐️

Сохраните название района с наименьшим числом поездок в переменную min_pickups, применив подходящий метод.
Исходные данные сохранены в переменную taxi


In [36]:
min_pickups = taxi.groupby('borough')['pickups'].sum().idxmin()

print(f"Район с наименьшим числом поездок: {min_pickups}")

Район с наименьшим числом поездок: EWR


#### Продолжим изучение данных и посмотрим на число поездок в выходные дни. 

Сгруппируйте данные по двум признакам: району города и является ли день выходным (колонки borough и hday). Сравните среднее число поездок, и выберите районы, из которых по праздникам в среднем поступает больше заказов, чем в обычные дни. 

In [37]:
# Создаем сводную таблицу
pivot_table = taxi.pivot_table(values='pickups', index='borough', columns='hday', aggfunc='mean')
print(pivot_table)

# Находим районы, где среднее в выходные больше
more_on_weekends = pivot_table[pivot_table['Y'] > pivot_table['N']].index.tolist()
print("Районы, где в выходные в среднем больше заказов:", more_on_weekends)

hday                     N            Y
borough                                
Bronx            50.771073    48.065868
Brooklyn        534.727969   527.011976
EWR               0.023467     0.041916
Manhattan      2401.302921  2035.928144
Queens          308.899904   320.730539
Staten Island     1.606082     1.497006
Районы, где в выходные в среднем больше заказов: ['EWR', 'Queens']


#### Для каждого района посчитайте число поездок по месяцам. 

Отсортируйте полученные значения по убыванию и сохраните результирующий датафрейм в pickups_by_mon_bor.
Обратите внимание, что итоговый датасет должен состоять из 3-х колонок - pickup_month, borough, pickups.
Данные сохранены в переменную taxi.


In [21]:
# Группируем по месяцу и району, считаем сумму поездок
pickups_by_mon_bor = taxi.groupby(['pickup_month', 'borough'])['pickups'].sum().reset_index()

# Сортируем по убыванию количества поездок
pickups_by_mon_bor = pickups_by_mon_bor.sort_values('pickups', ascending=False)

# Оставляем только нужные колонки
pickups_by_mon_bor = pickups_by_mon_bor[['pickup_month', 'borough', 'pickups']]

# Проверяем результат
print(pickups_by_mon_bor.head())

   pickup_month    borough  pickups
21          Jun  Manhattan  1995388
33          May  Manhattan  1888800
9           Feb  Manhattan  1718571
27          Mar  Manhattan  1661261
3           Apr  Manhattan  1648278


## 💀 Сложное задание! 💀

Теперь задача посложнее! Попрактикуемся в написании собственных функций и их применении к датафрейму. Поскольку данные о поездках в Нью-Йорке, температура представлена в градусах Фаренгейта. 
Напишите функцию temp_to_celcius, которая получает на вход колонку с температурой в °F, и возвращает значения, переведенные в градусы Цельсия.


NB! В окошке ниже надо написать только саму функцию. Так как урок посвящён pandas, то и всё содержимое функции должно быть выполнено средствами этой библиотеки - убедитесь, что на выходе она даёт pandas Series.

In [24]:
def temp_to_celcius(temp_f):
    return (temp_f - 32) * 5.0 / 9.0